Code to replicate the results reported for Study 2'.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
%load_ext rpy2.ipython
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.stats.api as sms
from pyspan.AFC.analysis import *
from pyspan.utils import *
from pyspan.valence import *

# Participants

In [3]:
# ddf contains data from Democrats
ddf = minidf.loc[minidf.party == "Democrat"]
# rdf contains data from Republicans
rdf = minidf.loc[minidf.party == "Republican"]

In [4]:
len(ddf), len(rdf)

(50, 19)

In [5]:
np.mean(ddf.age), stats.sem(ddf.age)

(40.66, 1.6229501985393067)

In [6]:
ddf.gender.value_counts()

1    30
0    20
Name: gender, dtype: int64

In [7]:
np.mean(rdf.age), stats.sem(rdf.age)

(43.8421052631579, 2.8789040697215458)

In [8]:
rdf.gender.value_counts()

1    10
0     9
Name: gender, dtype: int64

# Results

In [9]:
pos_words = [ "superior", "joy", "plentiful", "qualified", "laugh",  "clever",
              "rapid", "famous", "useful", "loyal"
            ]
neg_words = [ "inferior", "sorrow", "scarce", "unqualified", "cry", "stupid", "slow", 
              "unknown", "useless", "disloyal" 
            ]

In [10]:
antonyms = {
    "POSITIVE": pos_words,
    "NEGATIVE": neg_words
}

In [11]:
def get_valence_bias(i):
    party = minidf.loc[i]["party"].lower()
    condition = minidf.loc[i]["Condition"].lower()
    if party not in ("democrat", "republican"):
        return (np.nan, np.nan, np.nan)
    party = 1 if party == "republican" else 0
    condition = 1 if condition == "republican" else 0
    k = "POSITIVE" if party == condition else "NEGATIVE"
    valence_match = map(lambda w: w in antonyms[k], filter(lambda w: isinstance(w, str), 
                                                           minidf.loc[i][map(str, range(89, 99))]))
    
    return (np.mean(valence_match), party, condition)

In [12]:
arr = np.array(map(get_valence_bias, minidf.index))
valence_match = arr[:,0]
parties = arr[:,1]
conditions = arr[:,2]
ixs_not_nan = np.arange(len(valence_match))[~np.isnan(valence_match)]
valence_match = valence_match[ixs_not_nan]
parties = parties[ixs_not_nan]
conditions = conditions[ixs_not_nan]

In [13]:
x_dd = valence_match[(parties == 0) & (conditions == 0)]
x_dr = 1 - valence_match[(parties == 0) & (conditions == 1)]
x_rd = 1 - valence_match[(parties == 1) & (conditions == 0)]
x_rr = valence_match[(parties == 1) & (conditions == 1)]

In [14]:
len(x_dd), len(x_dr), len(x_rd), len(x_rr)

(24, 26, 12, 7)

Each observation in `x_dd` corresponds to a Democratic participant asked to choose the word most likely to have been spoken by a Democrat, and each observation in `x_rr` corresponds to a Republican participant asked to choose the word most likely to have been spoken by a Republican. Observations are the proportion of positive words the participant selected. 

In [15]:
np.mean(np.append(x_dd, x_rr)), stats.sem(np.append(x_dd, x_rr))

(0.8096774193548386, 0.02678261300448299)

Each observation in `x_dr` corresponds to a Democratic participant asked to choose the word most likely to have been spoken by a Republican, and each observation in `x_rd` corresponds to a Republican participant asked to choose the word most likely to have been spoken by a Democrat. Observations are the proportion of positive words the participant selected. 

In [16]:
np.mean(np.append(x_dr, x_rd)), stats.sem(np.append(x_dr, x_rd))

(0.3736842105263158, 0.032811416525782455)

In [17]:
prop_match = np.append(x_dd, x_rr)
prop_doesnt_match = np.append(x_dr, x_rd)

In [18]:
dsw_pm = sms.DescrStatsW(prop_match)
dsw_pdm = sms.DescrStatsW(prop_doesnt_match)
cm = sms.CompareMeans(dsw_pm, dsw_pdm)
cm.ttest_ind(usevar="unequal", alternative = "larger")

(10.293923175156955, 1.0890127910206365e-15, 66.38400483833256)

In [19]:
delta = dsw_pm.mean - dsw_pdm.mean
se_delta = cm.std_meandiff_separatevar
print(delta, delta - 2*se_delta, delta + 2*se_delta)

(0.4359932088285227, 0.3512843565455106, 0.5207020611115348)


## Mixed effects model

In [25]:
%%R -i minidf,pos_words,neg_words
library(lmerTest)
library(tidyverse)

options(contrasts=c("contr.sum","contr.poly"))

df <- minidf %>%
    as_tibble() %>%
    filter(party %in% c("Democrat","Republican")) %>%
    select(X89:X98, age, gender, party, party_identity, political_engagement, Condition) %>%
    mutate(ID=1:nrow(.)) %>%
    pivot_longer(X89:X98, names_to="item", values_to="pos") %>%
    filter(pos %in% c(pos_words, neg_words)) %>%
    mutate(pos=pos %in% pos_words) %>%
    mutate(party=ifelse(party=="Republican", sqrt(.5), -sqrt(.5)),
           Condition=ifelse(Condition=="REPUBLICAN", sqrt(.5), -sqrt(.5)),
           age=age-18,
           gender=ifelse(gender==1, -1, 1),
           party_identity=party_identity+3,
           political_engagement=political_engagement+3
          )
    
fit <- glmer(pos ~ Condition * party + (1|ID), data=df, family="binomial",
             glmerControl(optimizer="bobyqa", optCtrl = list(maxfun = 100000)
                         ))
summary(fit)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: pos ~ Condition * party + (1 | ID)
   Data: df
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 1e+05))

     AIC      BIC   logLik deviance df.resid 
   802.1    824.7   -396.0    792.1      685 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.9425 -0.7043  0.3806  0.5504  1.7052 

Random effects:
 Groups Name        Variance Std.Dev.
 ID     (Intercept) 0.2943   0.5425  
Number of obs: 690, groups:  ID, 69

Fixed effects:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)       0.6253     0.1414   4.423 9.71e-06 ***
Condition         0.2044     0.1990   1.027   0.3043    
party             0.3298     0.1990   1.657   0.0975 .  
Condition:party   2.2901     0.2862   8.001 1.24e-15 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr) Con

In [26]:
%%R
# +- 2 SE
coef.interact <- summary(fit)$coefficients[4,1]
se.interact <- summary(fit)$coefficients[4,2]
print(c(coef.interact - 2*se.interact, coef.interact + 2*se.interact))

[1] 1.717605 2.862535


In [27]:
%%R
exp(summary(fit)$coefficients[4,1])

[1] 9.87563


In [32]:
%%R
null.fit <- glmer(pos ~ Condition + party + (1|ID), data=df, family="binomial")
anova(null.fit, fit, test="LRT")

Data: df
Models:
null.fit: pos ~ Condition + party + (1 | ID)
fit: pos ~ Condition * party + Condition * age + Condition * gender + 
fit:     Condition * party * party_identity + Condition * party * 
fit:     political_engagement + (1 | ID) + (1 | item)
         npar    AIC    BIC  logLik deviance  Chisq Df Pr(>Chisq)    
null.fit    4 855.13 873.27 -423.56   847.13                         
fit        18 819.25 900.91 -391.62   783.25 63.877 14  2.424e-08 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [35]:
%%R
fit <- glmer(pos ~ Condition * party + Condition * age + Condition * gender + Condition * party * party_identity + Condition * party * political_engagement + (1|ID),
             data = df, family="binomial"
            )
summary(fit)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: pos ~ Condition * party + Condition * age + Condition * gender +  
    Condition * party * party_identity + Condition * party *  
    political_engagement + (1 | ID)
   Data: df

     AIC      BIC   logLik deviance df.resid 
   817.2    894.4   -391.6    783.2      673 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.7075 -0.7450  0.3742  0.5852  1.9732 

Random effects:
 Groups Name        Variance Std.Dev.
 ID     (Intercept) 0.1909   0.4369  
Number of obs: 690, groups:  ID, 69

Fixed effects:
                                      Estimate Std. Error z value Pr(>|z|)   
(Intercept)                           1.237585   0.381084   3.248  0.00116 **
Condition                             0.616052   0.538044   1.145  0.25222   
party                                -0.047210   0.512291  -0.092  0.92657   
age                                  -